<a href="https://colab.research.google.com/github/NyafeuKamdem/retail_demand_forcasting/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data_Prep**

##**Data Preparation Pipeline: From Drive to Clean Sample**

###**STEP 1: Import packages**

In [11]:
# Importing the needed packages.
import numpy as np                                                # For numerical operation arraywise.
import pandas as pd                                               # For Data manipulation.
import matplotlib.pyplot as plt                                   # For basic visualization's functions.
import seaborn as sns                                             # For advance visualization's functions.
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pwd  # pwd = 'print working directory' tels us where we are

/content


In [13]:
%cd drive/MyDrive/ # cd = 'change directory'

[Errno 2] No such file or directory: 'drive/MyDrive/ # cd = change directory'
/content


In [14]:
!pwd

/content


###**STEP 2: Read all files except train.csv**

In [102]:
# STEP 3: Read all files but train.csv

df_holiday_events = pd.read_csv('/content/drive/MyDrive/favorita/holidays_events.csv')
df_items          = pd.read_csv('/content/drive/MyDrive/favorita/items.csv')
df_oil            = pd.read_csv('/content/drive/MyDrive/favorita/oil.csv')
df_stores         = pd.read_csv('/content/drive/MyDrive/favorita/stores.csv')
df_transactions   = pd.read_csv('/content/drive/MyDrive/favorita/transactions.csv')

###**STEP 3:Load stores and get Guayas store IDs**

In [103]:
store_ids = df_stores[df_stores['state'] == 'Guayas']['store_nbr'].unique()

###**STEP 4: Read train.csv in chunks and filter for Guayas region**

In [104]:
# STEP 6: Read train.csv in chunks and filter for Guayas region
chunk_size = 10**6
filtered_chunks = []

for chunk in pd.read_csv('/content/drive/MyDrive/favorita/train.csv', chunksize = chunk_size):
    chunk_filtered = chunk[chunk['store_nbr'].isin(store_ids)]
    filtered_chunks.append(chunk_filtered)
    del chunk

/tmp/ipython-input-3821781864.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('/content/drive/MyDrive/favorita/train.csv', chunksize = chunk_size):


###**STEP 5: Combine and sample**

In [105]:
# STEP 5: Combine and sample
df_train = pd.concat(filtered_chunks, ignore_index=True).sample(n=2_000_000).reset_index(drop=True)

In [106]:
df_train.shape

(2000000, 6)

###**STEP 6: Clean up memory and save df_train as a pickle file**

In [107]:
len(filtered_chunks)
del filtered_chunks

In [108]:
df_items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [109]:
# top1 family with 'perishable' items

df_items_perish = df_items[df_items['perishable']==1]
top1_family_per_perish_item = df_items_perish['family'].value_counts().reset_index()
top1_family_per_perish_item

,family,count
0,PRODUCE,306
1,DAIRY,242
2,BREAD/BAKERY,134
3,DELI,91
4,MEATS,84
5,POULTRY,54
6,EGGS,41
7,PREPARED FOODS,26
8,SEAFOOD,8


In [110]:
# SELECT THE TOP 3 FAMILIES(IN TERMS OF NUMBER OF ITEM) + A FAMILY OF 'PERISHABLE' ITEMS

top3_family_per_item = df_items['family'].value_counts().reset_index()
top3_family_per_item = top3_family_per_item.rename(columns = {'count': 'item quantity'}, inplace = False)
top3_family_per_item = top3_family_per_item[:3] # top3_family_per_item = top3_family_per_item.head(3)
top3_family_per_item

,family,item quantity
0,GROCERY I,1334
1,BEVERAGES,613
2,CLEANING,446


In [111]:
families = top3_family_per_item['family'].unique().tolist()
families.append(top1_family_per_perish_item['family'][0])
families

['GROCERY I', 'BEVERAGES', 'CLEANING', 'PRODUCE']

In [113]:
# CREATE THE LIST OF ITEM'S IDS AND USE IT TO FILTER THE DATA FRAME.

item_ids = [x for x in df_items[df_items['family'].isin(families)]['item_nbr']]

df_train = df_train[df_train['item_nbr'].isin(item_ids)]
df_train.shape

(1398849, 6)

In [114]:
# Path to save the file
save_path = "/content/drive/MyDrive/tsa_ms_project/df_train.pkl"

# Save DataFrame
df_train.to_pickle(save_path)
print(f"Saved to: {save_path}")

Saved to: /content/drive/MyDrive/tsa_ms_project/df_train.pkl


In [4]:
from google.colab import drive
import pandas as pd

# Mount Drive
drive.mount('/content/drive')

# Path to load the file
load_path = "/content/drive/MyDrive/tsa_ms_project/df_train.pkl"

# Load DataFrame
df_train = pd.read_pickle(load_path)
print("DataFrame reloaded, ready to go!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DataFrame reloaded, ready to go!
